In [ ]:
!pip install --upgrade tensorflow
!pip install --upgrade keras
!pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 546.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 79.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.5 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.8.0
    Uninstalling keras-3.8.0:
      Successfully uninstalled keras-3.8.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 13.7 MB/s eta 0:00:00


In [ ]:
import os
import cv2
import numpy as np
from skimage import filters
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet101, ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import BinaryCrossentropy
import numpy as np
import cv2
import os
from tensorflow.keras.metrics import MeanIoU
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Define the folder path for your data
image_dir = '/content/drive/My Drive/Output/images'  # Update this path
label_dir = '/content/drive/My Drive/Output/labels'

In [ ]:
# 1. Load the images and labels from the directory
def load_images_and_labels(image_dir, label_dir):
    image_files = [f for f in os.listdir(image_dir) if f.endswith('.jpg')]
    label_files = [f for f in os.listdir(label_dir) if f.endswith('.png')]

    image_files.sort()
    label_files.sort()

    images = []
    labels = []

    for image_file, label_file in tqdm(zip(image_files, label_files), total=len(image_files), desc="Loading images and labels"):
        img_path = os.path.join(image_dir, image_file)
        label_path = os.path.join(label_dir, label_file)

        # Read image and label
        img = cv2.imread(img_path)
        label = cv2.imread(label_path, cv2.IMREAD_GRAYSCALE)

        # Resize image and label to 256x256
        img = cv2.resize(img, (256, 256))
        label = cv2.resize(label, (256, 256))

        # Normalize image (0-1)
        img = img.astype(np.float32) / 255.0

        # Convert label to binary (0 or 1)
        label = (label > 127).astype(np.uint8)

        images.append(img)
        labels.append(label)

    return np.array(images), np.array(labels)


In [ ]:
# U-Net model with ResNet50 backbone
def unet_resnet50(input_size=(256, 256, 3)):
    # Load the ResNet50 backbone without the top layer
    base_model = ResNet101(weights='imagenet', include_top=False, input_shape=input_size)

    # Encoder: ResNet50 layers
    encoder_output = base_model.output
    encoder_output = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(encoder_output)

    # Decoder: Upsampling layers
    up1 = layers.Conv2DTranspose(256, (3, 3), strides=(2, 2), padding='same')(encoder_output)
    up1 = layers.Concatenate()([up1, base_model.get_layer('conv4_block6_out').output])

    up2 = layers.Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same')(up1)
    up2 = layers.Concatenate()([up2, base_model.get_layer('conv3_block4_out').output])

    up3 = layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same')(up2)
    up3 = layers.Concatenate()([up3, base_model.get_layer('conv2_block3_out').output])

    up4 = layers.Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same')(up3)
    up4 = layers.Concatenate()([up4, base_model.get_layer('conv1_relu').output])

    # Final Output Layer: 1 channel for binary classification (background or object)
    output = layers.Conv2D(1, (1, 1), activation='sigmoid')(up4)

    # Upsample output to match the input size (256x256)
    output = layers.UpSampling2D(size=(2, 2), interpolation='bilinear')(output)

    # Build the model
    model = models.Model(inputs=base_model.input, outputs=output)

    return model

In [ ]:
# 3. Split the data into training and testing sets
def split_data(images, labels, test_size=0.2):
    # Use train_test_split to split the data
    X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=test_size, random_state=42)

    print(f"Training data shape: {X_train.shape}")
    print(f"Testing data shape: {X_test.shape}")
    return X_train, X_test, y_train, y_test

In [ ]:
images, labels = load_images_and_labels(image_dir, label_dir)

Loading images and labels: 100%|██████████| 6226/6226 [36:42<00:00,  2.83it/s]


In [ ]:
# 5. Split the data into training and testing sets
X_train, X_test, y_train, y_test = split_data(images, labels)

In [ ]:
X_train[0]

In [ ]:
y_train[1]

In [ ]:
print(X_train.shape, X_train.dtype)
print(X_test.shape,X_test.dtype)
print(y_train.shape,y_train.dtype)
print(y_test.shape,y_test.dtype)

In [ ]:
y_train = np.expand_dims(y_train, axis=-1)
y_test = np.expand_dims(y_test, axis=-1)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

In [ ]:
y_test[20]

In [ ]:
# Plot the actual image (X_test[0])
plt.figure(figsize=(6, 6))
plt.imshow(y_test[0])  # Display the original image
plt.title("Actual Image (X_test[0])")
plt.axis('off')  # Hide the axis
plt.show()

In [ ]:
print(X_train.shape, X_train.dtype)
print(X_test.shape,X_test.dtype)
print(y_train.shape,y_train.dtype)
print(y_test.shape,y_test.dtype)

In [ ]:
import tensorflow as tf

def dice_loss(y_true, y_pred):
    smooth = 1e-6  # Avoid division by zero
    intersection = tf.reduce_sum(y_true * y_pred)
    return 1 - (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

In [ ]:
# model = unet_model(input_shape=(256, 256, 3))
# model.compile(optimizer=Adam(learning_rate=0.001),
#               loss=BinaryCrossentropy(),
#               # metrics=[MeanIoU(num_classes=2)]
#               metrics=['accuracy'])

# Create the U-Net model
model = unet_resnet101(input_size=(256, 256, 3))

# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-4), loss=dice_loss, metrics=[MeanIoU(num_classes=2)])

In [ ]:
# ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Fit the generator on your data
datagen.fit(X_train)

In [ ]:
# # Reshaping the y_train to (256, 256, 1)
# y_train = y_train[..., np.newaxis]

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
# Callbacks
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ModelCheckpoint('unet_resnet50_best_model.h5', save_best_only=True)
]

# Train the model with callbacks
history = model.fit(datagen.flow(X_train, y_train, batch_size=16),
                    epochs=10,
                    steps_per_epoch=5,
                    callbacks=callbacks,validation_data=(X_test, y_test))

In [ ]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

In [ ]:
predictions = model.predict(X_test)

NameError: name 'model' is not defined

In [ ]:
print(predictions[0].shape)

In [ ]:
print(X_test[0].shape)

In [ ]:
predictions[0]

In [ ]:
!pip install rasterio

In [ ]:
import rasterio
import matplotlib.pyplot as plt


In [ ]:
# Plot the actual image (X_test[0])
plt.figure(figsize=(6, 6))
plt.imshow(predictions[0])  # Display the original image
plt.title("Actual Image (X_test[0])")
plt.axis('off')  # Hide the axis
plt.show()

In [ ]:
# Plot the actual image (X_test[0])
plt.figure(figsize=(6, 6))
plt.imshow(y_test[20])  # Display the original image
plt.title("Actual Image (X_test[0])")
plt.axis('off')  # Hide the axis
plt.show()

In [ ]:
y_test[20]

In [ ]:
predicted_mask = (predictions[10] > 0.5).astype(np.uint8)
predicted_mask


In [ ]:
# Plot the actual image (X_test[0])
plt.figure(figsize=(6, 6))
plt.imshow(predicted_mask)  # Display the original image
plt.title("Actual Image (X_test[0])")
plt.axis('off')  # Hide the axis
plt.show()

In [ ]:
pred=model.predict(X_test)

In [ ]:
pred[0]

In [ ]:
y_test[0]

In [ ]:
from sklearn.metrics import jaccard_score

thresholded_predictions = predicted_mask
actual_array = y_test[10]
# Flatten the arrays to compute IoU (Intersection over Union)
flat_predictions = thresholded_predictions.flatten()
flat_labels = actual_array.flatten()

iou = jaccard_score(flat_labels, flat_predictions)
print(f"Intersection over Union (IoU): {iou}")